In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchsummary import summary

class SNeurodCNN(nn.Module):
    def __init__(self):
        super(SNeurodCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=0)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.conv2 = nn.Conv2d(32, 32, kernel_size=3, stride=1, padding=0)
        self.conv3 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=0)
        self.adaptive_pool = nn.AdaptiveAvgPool2d((44, 44))
        self.fc1 = nn.Linear(64 * 44 * 44, 500)
        self.dropout = nn.Dropout(0.5)
        self.fc2 = nn.Linear(500, 2)
        
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = F.relu(self.conv2(x))
        x = self.adaptive_pool(F.relu(self.conv3(x)))
        x = x.view(-1, 64 * 44 * 44) 
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

    
model = SNeurodCNN()

print(model)
summary(model, input_size=(1, 180, 180))

SNeurodCNN(
  (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1))
  (conv3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
  (adaptive_pool): AdaptiveAvgPool2d(output_size=(44, 44))
  (fc1): Linear(in_features=123904, out_features=500, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc2): Linear(in_features=500, out_features=2, bias=True)
)
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 32, 178, 178]             320
         MaxPool2d-2           [-1, 32, 89, 89]               0
            Conv2d-3           [-1, 32, 87, 87]           9,248
            Conv2d-4           [-1, 64, 85, 85]          18,496
 AdaptiveAvgPool2d-5           [-1, 64, 44, 44]               0
            Linear-6                  [-1, 

In [2]:
# Optimizer: Adam
# Learning rate: 0.0001
# Epochs: 100
# Batch size: 32
# Regularizers: Early stopping (patience = 5, restore_best_weights = True)